# Physics 5700 final project
### Noah Myers: 4/20/23

The final project for physics 5700 which includes a solution of the Double pendulum problem using the Euler-Lagrange equation as well as solving a two body gravitational orbit problem in Cartesian coordinates

### 1: Multi-pendulum using Lagrange's equation

Defines a doublependulum class that is used to generate basic pendulum plots from solving Lagrange's equations.


## Double Pendulem equation. 
 
For a double pendulum, we can look at (11.37) to find the potential energy U($\phi_1, \phi_2$) and (11.38) to find Kinetic Energy T. 


We find the potential energy is,

$\begin{align}
{U} = -(m_1+m_2)gL_1(1 - \cos\phi_1) -m_2gL_2(1 - \cos\phi_2)
\end{align}$    (11.37)

and the kinetic energy is, 

$\begin{align}
{T} = \frac12 (m_1+m_2) L_1^2 \dot\phi_1^2 + m_2 L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_1-\phi_2) +\frac12 m_2 L_2^2 \dot\phi_2^2
\end{align}$    (11.38)


## Euler-Lagrange equations for double pendulum

In general the Largrangian equation is given by, 
$\begin{align}
  \mathcal{L} = {T} - {U}
\end{align}$
So for this double pendulum, we will have the Larangian with generalized coordinates $\phi_1$ and $\phi_2$ which is, 

$\begin{align}
  \mathcal{L} = \frac12 (m_1+m_2) L_1^2 \dot\phi_1^2 + m_2 L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_1-\phi_2) +\frac12 m_2 L_2^2 \dot\phi_2^2 +(m_1+m_2)gL_1(1 - \cos\phi_1) +m_2gL_2(1 - \cos\phi_2)
\end{align}$

The Euler-Lagrange equations are

$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_i} = \frac{\partial\mathcal L}{\partial\phi_i}
 \mbox{for}  \phi_i = \phi_1, \phi_2
  \;.
\end{align}$

For these coordinates after some calculs and algebra, we get 



$\begin{align}
(m_1+m_2) L_1 \ddot\phi_1 + m_2L_2\ddot\phi_2\cos(\phi_1-\phi_2) +m_2 L_2 \ddot\phi_2^2\sin(\phi_1-\phi_2) +(m_1+m_2)g( \sin\phi_1) = 0 
\end{align}$
$\begin{align}
m_2 L_2 \ddot\phi_2 + m_2L_1\ddot\phi_1\cos(\phi_1-\phi_2) -m_2 L_1 \ddot\phi_1^2\sin(\phi_1-\phi_2) +m_2g( \sin\phi_2) = 0 
\end{align}$

integrate.solve_ivp will only work if given first-order differential equations, so we will need to redefine $\dot\phi_1, \ddot\phi_1, \dot\phi_2, \ddot\phi_2$, so we will let ${x_1} \equiv \dot\phi_1 => {\dot x_1} = \ddot\phi_1, {x_2} \equiv \dot\phi_2 => \dot{x_2} = \ddot\phi_2$. This will lead to the equations, 

$\begin{align}
 \dot{x_1} = \frac{m_2g\sin\phi_2\cos(\phi_1 - \phi_2) - m_2\sin(\phi_1-\phi_2)[L_1 x_1^2\cos(\phi_1-\phi_2) + L_2x_2^2] - (m_1+m_2)g\sin\phi_1}{L_1[m_1+m_2\sin^2(\phi_1-\phi_2)]}
 \end{align}$
 
$\begin{align}
 \dot{x_2} = \frac{(m_1+m_2)[L_1x_1^2\sin(\phi_1-\phi_2) - g\sin\phi_2 + g\sin\phi_1\cos(\phi_1-\phi_2)] + m_2L_2x_2^2\sin(\phi_1-\phi_2)\cos(\phi_1-\phi_2)}{L_2[m_1+m_2\sin^2(\phi_1-\phi_2)]}
 \end{align}$


In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy.integrate import odeint, solve_ivp

import matplotlib.pyplot as plt
from matplotlib import animation , rc 
from IPython.display import HTML

In [3]:
class doublependulum():
    """
    doublependulum class inputs the parameters and lagrangian equations for double pendulum with no driving or damping. 
    
    Parameter 
    ---------
    g: float
    gravitational acceleration
    m_n: float 
    mass of the pendulum, with n telling which mass on the pendulum
    L_n:float 
    Lenght of the pendulums with n telling which pendulum 
    
    
    Methods
    ----------
    dy_dt(t,y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    
    def __init__(self, L_1=1, L_2=1, m_1 = 1, m_2=1, g=1):
        self.L_1 = L_1
        self.L_2 = L_2
        self.m_1 = m_1
        self.m_2 = m_2
        self.g = g 
        
    def dy_dt(self,t,y):
        """
        this returns the right hand side of the differential equation: 
        phi_dot_1, phi_ddot_1, phi_dot_2, phi_ddot_2
        
        Parameters
        -----------
        
        y: float
        4 comp vector 
        
        y[0]= phi_1, y[1]= phi_dot_1, y[2] = phi_2, y[3] = phi_dot_3
        """
        
        phi_1 ,x_1, phi_2, x_2 = y
        
        phi_dot_1 = x_1
        phi_dot_2 = x_2
        
        c,s = np.cos(phi_1 - phi_2), np.sin(phi_1 - phi_2)
        
        denom = (self.m_1 + self.m_2 * s**2)
        
        x_dot_1 = (self.m_2 * self.g * np.sin(phi_2)* c -self.m_2 *s *(self.L_1 * x_1**2 *c + self.L_2 * x_2**2)
                   -(self.m_1 + self.m_2) * self.g * np.sin(phi_1)) /self.L_1/denom
        x_dot_2 = ((self.m_1 + self.m_2) * (self.L_1 * x_1**2 * s - self.g*np.sin(phi_2) +self.g * np.sin(phi_1) *c) 
                   + self.m_2 *self.L_2 * x_2**2 *s *c) /self.L_2/denom
        return phi_dot_1, x_dot_1, phi_dot_2, x_dot_2
    
    def solve_ode(self, t_pts, phi_1_0, phi_dot_1_0, phi_2_0, phi_dot_2_0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi_1_0, phi_dot_1_0, phi_2_0, phi_dot_2_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi_1, phi_dot_1, phi_2, phi_dot_2= solution.y

        return  phi_1, phi_dot_1, phi_2, phi_dot_2
    
    def E(y):
        '''
        Returns the total energy of the system
        '''
        
        p1, pd1, p2,pd2 = y.T
        V = -(m_1+m_2)* L_1 *g*np.cos(p1)- m_2*L_2*g*np.cos(p2)
        T = 1/2 * m_1 *(L_1*pd1)**2 + 1/2 * m_2*((L_1*pd1)**2 + (L_2 *pd2)**2 + 2*L_1*L_2*pd1*pd2*np.cos(p1-p2))
        E_tot = T+V
        return E_tot

In [4]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [5]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Initializing inital conditions and plotting double pendulum motion for each mass

In [6]:
#Labels for individual plot axes
phi_vs_time_labels = (r'$t$' , r'$\phi(t)$')

#Setting up plotting time slices 
t_start = 0
t_end = 50
delta_t = .001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)

#inital conditions for mass and lenght
L_1 = 1
L_2 =1
m_1 = 1
m_2 =1 
g =1 

#applying the double pendulum class 

dp_1 = doublependulum(L_1 =L_1, L_2= L_2, m_1 = m_1, m_2 = m_2, g=g)

In [ ]:
#inital conditions of phi and phi_dot for each pendulum 

phi_1_0 = np.pi/2
phi_dot_1_0 = 0 
phi_2_0 = np.pi
phi_dot_2_0 = 0 

phi_1, phi_dot_1, phi_2, phi_dot_2 = dp_1.solve_ode(t_pts,phi_1_0, phi_dot_1_0, phi_2_0, phi_dot_2_0)


# start the plot!
fig = plt.figure(figsize=(15,5))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf' $\phi_1(0) = {phi_1_0:.2f},$' + \
                rf'  $\dot\phi_1(0) = {phi_dot_1_0:.2f},$' + \
                rf' $\phi_2(0) = {phi_2_0:.2f},$' + \
                rf'  $\dot\phi_2(0) = {phi_dot_2_0:.2f},$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot 
ax_a = fig.add_subplot(1,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi_1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue', 
            label=r'$\phi_1(t)$', 
            ax=ax_a)      
plot_y_vs_x(t_pts[start : stop], phi_2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            ax=ax_a)    
                                                    


(<AxesSubplot:xlabel='$t$', ylabel='$\\phi(t)$'>,
 <matplotlib.lines.Line2D at 0x247080fa070>)

## Showing Chaos using small variation in inital conditions. 

In [ ]:
phi_1_0 = np.pi/2
phi_dot_1_0 = 0 
phi_2_0 = np.pi
phi_dot_2_0 = 0 

t_start = 0
t_end = 400
delta_t = .05

t_pts1 = np.arange(t_start, t_end+delta_t, delta_t)


#same initial conditions used in above plots
phi_1, phi_dot_1, phi_2, phi_dot_2 = dp_1.solve_ode(t_pts1,phi_1_0, phi_dot_1_0, phi_2_0, phi_dot_2_0)

#slightly vary inital conditions to show chaos
phi_1a, phi_dot_1a, phi_2a, phi_dot_2a = dp_1.solve_ode(t_pts1,phi_1_0, phi_dot_1_0, phi_2_0-.00001, phi_dot_2_0)


# start the plot!
fig = plt.figure(figsize=(15,5))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf' $\phi_1(0) = {phi_1_0:.2f},$' + \
                rf'  $\dot\phi_1(0) = {phi_dot_1_0:.2f},$' + \
                rf' $\phi_2(0) = {phi_2_0:.2f},$' + \
                rf'  $\dot\phi_2(0) = {phi_dot_2_0:.2f},$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot 
ax_a = fig.add_subplot(1,1,1)                  
#plotting the diffearnce between either variable and its counterpart with small variation
start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts1[start : stop], np.abs(phi_1[start : stop] - phi_1a[start:stop]), 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            semilogy = True,
            label=r'$\Delta\phi_1(t)$', 
            ax=ax_a)      
plot_y_vs_x(t_pts1[start : stop], np.abs(phi_2[start : stop]- phi_2a[start:stop]), 
            axis_labels=phi_vs_time_labels, 
            color='red',
            semilogy = True,
            label=r'$\Delta\phi_2(t)$', 
            ax=ax_a)    
                                                    


We can see that as time increases the differance between the $\phi_1$ and $\phi_2$ grows. This growth becomes uncontrolable and eventually would make these two plots of the two cases with very small yet different inital conditions very different from eachother. This shows that this double pendulum set up is chaotic outside of the small angles and that small variations in inital conditions lead to wildly different results

## Animation!!

In [ ]:
def xy (x0, y0, phi, L): 
    """
    this will convert the phi angle and leght L of the pendulum into x,y corrdinate postion
    """
    x = x0 + L *np.sin(phi)
    y = y0 - L *np.cos(phi)
    return x,y

In [ ]:
%%capture 
x_min = -3.2 
x_max = -x_min 
y_min = -3.2 
y_max = -y_min 

fig_anim = plt.figure(figsize = (8,8), num = 'Double pendulum')
ax_anim = fig_anim.add_subplot(1,1,1)
ax_anim.set_xlim(x_min, x_max)
ax_anim.set_ylim(y_min, y_max)

#assign the first return from plot to pt1_anim and others so we can later change the values 

x0,y0 = 0.,0.
pt0_anim, = ax_anim.plot(x0,y0, 'o', markersize =6, color = 'black')


x1, y1 = xy(x0,y0, phi_1[0], dp_1.L_1)
pt1_anim, = ax_anim.plot(x1,y1,'o', markersize =12, color = 'b')
ln1_anim, = ax_anim.plot([x0,x1], [y0,y1], color = 'b', lw = 3)

x2, y2 = xy(x1,y1, phi_2[0], dp_1.L_2)
pt2_anim, = ax_anim.plot(x2,y2,'o', markersize =12, color = 'b')
ln2_anim, = ax_anim.plot([x1,x2], [y1,y2], color = 'b', lw = 3)

x1a, y1a = xy(x0,y0, phi_1a[0], dp_1.L_1)
pt1a_anim, = ax_anim.plot(x1a,y1a,'o', markersize =12, color = 'r')
ln1a_anim, = ax_anim.plot([x0,x1a], [y0,y1a], color = 'r', lw = 3)


x2a, y2a = xy(x1a,y1a, phi_2a[0], dp_1.L_2)
pt2a_anim, = ax_anim.plot(x2a,y2a,'o', markersize =12, color = 'r')
ln2a_anim, = ax_anim.plot([x1a,x2a], [y1a,y2a], color = 'r', lw = 3)


ax_anim.set_aspect(1)
ax_anim.axis('off')
fig_anim.tight_layout()



In [ ]:
def animate(i): 
    """
    This function will create each from, numbered by i. 
    So each i corresponds to a point in the t_pts array with index i. 
    """
    
    i_skip = 2*i 
    
    x0,y0 = 0., 0.
    pt0_anim.set_data(x0,y0)


    x1, y1 = xy(x0,y0, phi_1[i_skip], dp_1.L_1)
    pt1_anim.set_data(x1,y1)
    ln1_anim.set_data([x0,x1], [y0,y1])
    
    x2, y2 = xy(x1,y1, phi_2[i_skip], dp_1.L_2)
    pt2_anim.set_data(x2,y2)
    ln2_anim.set_data([x1,x2], [y1,y2])

    x1a, y1a = xy(x0,y0, phi_1a[i_skip], dp_1.L_1)
    pt1a_anim.set_data(x1a,y1a)
    ln1a_anim.set_data([x0,x1a], [y0,y1a])


    x2a, y2a = xy(x1a,y1a, phi_2a[i_skip], dp_1.L_2)
    pt2a_anim.set_data(x2a,y2a)
    ln2a_anim.set_data([x1a,x2a], [y1a,y2a])
    
    return (pt0_anim, pt1_anim, ln1_anim, pt2_anim, ln2_anim,
           pt1a_anim, ln1a_anim, pt2a_anim, ln2a_anim)

In [ ]:
frame_interval = 20 #time between frames
frame_number = 1001 #number of frames

anim = animation.FuncAnimation(fig_anim, animate, init_func = None, 
                              frames = frame_number, 
                              interval = frame_interval, 
                              blit = True, 
                              repeat = False)

In [ ]:
HTML(anim.to_jshtml())

## 2: Gravitational orbits in Cartesian coordinates
Deinfs a gravitationalorbits calss that will be able to calculate orbits due to gravity and convert them into Cartesian coordinates 

In [ ]:
class gravitationalorbits():
    """
    this class implements various parameters for gravitational orbits aswell as 
    Lagrange's equations for two bodys orbiting under attratction due to gravity.
    
    Parameters
    --------
    m_1: float
        mass of body 1 
    m_2: float 
        mass of body 2
    G: float 
        gravitational constant
        
    Methods
    ---------
    dy_dt(t,y)
    Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, G =1 , m_1 = 1 , m_2 = 1):
        self.G = G
        self.m_1 = m_1 
        self.m_2 = m_2 
    def dz_dt(self,t, z): 
        '''
        this function returns the right hand side of the differential equation: 
        [dz/dt , d^2z/dt^2]
        
        paramerts
        ----------
        t = float 
            time 
        z = float
            8 comp vector with 
             z[0]= x1(t), z[2]= y1(t), z[4] = x2(t), z[6] = y2(t)
             z[1]= x_dot1(t), z[3]= y_dot1(t), z[5] = x_dot1(t), z[7] = y_dot2(t)
        '''
        
        #radius of two body orbit being converted into x,y coordinates 
        
        r12 = np.sqrt((z[0] - z[4])**2 + (z[2] - z[6])**2)
        
        return [z[1], self.G *self.m_2 * (z[4]- z[0])/r12 **3, 
               z[3], self.G *self.m_2 * (z[6]- z[2])/r12 **3, 
               z[5], -self.G *self.m_1 * (z[4]- z[0])/r12 **3, 
               z[7], -self.G *self.m_1 * (z[6]- z[2])/r12 **3]
    
    
    def solve_ode(self, t_pts, z0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
         
        solution = solve_ivp(self.dz_dt, (t_pts[0], t_pts[-1]), 
                             z0, t_eval=t_pts, method = 'RK23',
                             atol=abserr, rtol=relerr)
        x1, x_dot1, y1, y_dot1, x2, x_dot2, y2, y_dot2 =  solution.y

        return  x1, x_dot1, y1, y_dot1, x2, x_dot2, y2, y_dot2
    def  leapfrog_solve_ode(self, t_pts, z0): 
        """
        Solve the ODE given initial conditions using leapfrog method
        """
        delta_t = t_pts[1]- t_pts[0]
        x10, x_dot10, y10, y_dot10, x20, x_dot20, y20, y_dot20 = z0
        
        #get an arry with zeros 
        num_t_pts = len(t_pts)
        
        x1 = np.zeros(num_t_pts)
        x_dot1 = np.zeros(num_t_pts)
        x_dot1_half = np.zeros(num_t_pts)
        
        y1 = np.zeros(num_t_pts)
        y_dot1 = np.zeros(num_t_pts)
        y_dot1_half = np.zeros(num_t_pts)
        
        x2 = np.zeros(num_t_pts)
        x_dot2 = np.zeros(num_t_pts)
        x_dot2_half = np.zeros(num_t_pts)
        
        y2 = np.zeros(num_t_pts)
        y_dot2 = np.zeros(num_t_pts)
        y_dot2_half = np.zeros(num_t_pts)
        
        #inital coordinates
        x1[0]= x10
        x_dot1[0]= x_dot10
        
        y1[0]= y10
        y_dot1[0]= y_dot10
        
        x2[0]= x20
        x_dot2[0]= x_dot20
        
        y2[0]= y20
        y_dot2[0]= y_dot20
        
        #now we need to take steps in the diffeq 
        for i in np.arange(num_t_pts -1): 
            t = t_pts[i]
            
            z = [x1[i], x_dot1[i], y1[i], y_dot1[i], 
                x2[i], x_dot2[i], y2[i], y_dot2[i]]
            out = self.dz_dt(t,z)
            
            x_dot1_half[i] = x_dot1[i] +out[1] * delta_t/2
            x1[i+1] = x1[i] + x_dot1_half[i] *delta_t 
            
            y_dot1_half[i] = y_dot1[i] +out[3] * delta_t/2
            y1[i+1] = y1[i] + y_dot1_half[i] *delta_t 
            
            x_dot2_half[i] = x_dot2[i] +out[5] * delta_t/2
            x2[i+1] = x2[i] + x_dot2_half[i] *delta_t 
            
            y_dot2_half[i] = y_dot2[i] +out[7] * delta_t/2
            y2[i+1] = y2[i] + y_dot2_half[i] *delta_t 
            
            z = [x1[i+1], x_dot1[i+1], y1[i+1], y_dot1[i+1], 
                x2[i+1], x_dot2[i+1], y2[i+1], y_dot2[i+1]]
            out = self.dz_dt(t,z)
            
            x_dot1[i+1] = x_dot1_half[i] + out[1] * delta_t/2
            y_dot1[i+1] = y_dot1_half[i] + out[3] * delta_t/2
            x_dot2[i+1] = x_dot2_half[i] + out[5] * delta_t/2
            y_dot2[i+1] = y_dot2_half[i] + out[7] * delta_t/2
        
        return x1, x_dot1, y1, y_dot1,x2, x_dot2,y2, y_dot2
            
    def energy(self, t_pts, x1, x_dot1, y1, y_dot1,x2, x_dot2,y2, y_dot2):
        """Evaluate the energy as a function of time using U(r) = -Gm_1m_2/r12
        Parameters
        ---------
        All x and y parameters for position
        """
        
        r12 = np.sqrt((x1-x2)**2 + (y1-y2)**2)
        U_r = -(self.G *self.m_1 * self.m_2) / (r12)
        
        r12_dot = ((x1-x2)*(x_dot1-x_dot2) + (y1-y2)*(y_dot1-y_dot2))/(r12)
        mu = (self.m_1 * self.m_2)/(self.m_1 + self.m_2)
        return (mu/2.) * r12_dot**2 + U_r
        

## Simple Orbit plots



In [ ]:
#labels for plots 
orbit_labels = (r'$x$', r'$y$')

#plotting times 
t_start = 0 
t_end = 20
delta_t = .0001


t_pts = np.arange(t_start, t_end+delta_t, delta_t)

#parameters 

G =20
m_1 = 20
m_2 = 1

o1 = gravitationalorbits(G, m_1, m_2)

#inital conditions with center of mass velocity = 0 
x10, x_dot10 = .1, 0
y10, y_dot10 = 0, .75
x20, x_dot20 = -(m_1/m_2)* x10, -(m_1/m_2)*x_dot10
y20, y_dot20 = -(m_1/m_2)* y10, -(m_1/m_2)*y_dot10

z0 = [x10, x_dot10, y10, y_dot10
     ,x20, x_dot20, y20, y_dot20]
x1_ode, x_dot1_ode, y1_ode, y_dot1_ode, x2_ode, x_dot2_ode, y2_ode, y_dot2_ode = o1.solve_ode(t_pts, z0)


#plotting 

fig = plt.figure(figsize = (8,8))
overall_title = 'Simple Gravitational Orbit'
fig.suptitle(overall_title, va='baseline')

ax =fig.add_subplot(1,1,1)

start,stop = start_stop_indices(t_pts, t_start, t_end)

ax.plot(x1_ode,y1_ode, color ='b', label =r'$m_1$')
ax.plot(x2_ode,y2_ode, color ='r', label =r'$m_2$')
ax.legend()
ax.set_aspect(1)

fig.tight_layout()


## Simple Orbit using Leapfrog Method

In [ ]:
t_start = 0 
t_end = 20
delta_t = .0001


t_pts = np.arange(t_start, t_end+delta_t, delta_t)

#parameters 

G =20
m_1 = 20
m_2 = 1

o2 = gravitationalorbits(G, m_1, m_2)

#inital conditions with center of mass velocity = 0 
x10, x_dot10 = .1, 0
y10, y_dot10 = 0, .75
x20, x_dot20 = -(m_1/m_2)* x10, -(m_1/m_2)*x_dot10
y20, y_dot20 = -(m_1/m_2)* y10, -(m_1/m_2)*y_dot10

z0 = [x10, x_dot10, y10, y_dot10
     ,x20, x_dot20, y20, y_dot20]
x1_lf, x_dot1_lf, y1_lf, y_dot1_lf, x2_lf, x_dot2_lf, y2_lf, y_dot2_lf = o2.leapfrog_solve_ode(t_pts, z0)


#plotting 

fig = plt.figure(figsize = (8,8))
overall_title = 'Simple Gravitational Orbit using LeapFrog'
fig.suptitle(overall_title, va='baseline')

ax =fig.add_subplot(1,1,1)

start,stop = start_stop_indices(t_pts, t_start, t_end)

ax.plot(x1_lf,y1_lf, color ='b', label =r'$m_1$')
ax.plot(x2_lf,y2_lf, color ='r', label =r'$m_2$')
ax.legend()
ax.set_aspect(1)

fig.tight_layout()

## Comparing the energy of Solve ODE RK 23 vs Leap Frog

In [ ]:
E_tot_pts_LF = o2.energy(t_pts, x1_lf, x_dot1_lf, y1_lf, y_dot1_lf, x2_lf, x_dot2_lf, y2_lf, y_dot2_lf)
E_tot_0_LF = E_tot_pts_LF[0]
E_tot_rel_pts_LF = np.abs((E_tot_pts_LF - E_tot_0_LF)/E_tot_0_LF)

E_tot_pts_ode = o1.energy(t_pts, x1_ode, x_dot1_ode, y1_ode, y_dot1_ode, x2_ode, x_dot2_ode, y2_ode, y_dot2_ode)
E_tot_0_ode = E_tot_pts_ode[0]
E_tot_rel_pts_ode = np.abs((E_tot_pts_ode - E_tot_0_ode)/E_tot_0_ode)

In [ ]:
fig_5 = plt.figure(figsize=(6,6))

overall_title = 'Energy comparison of Rk23 vs Leapfrog'
fig_5.suptitle(overall_title, va='baseline')

ax_5a = fig_5.add_subplot(1,1,1)
#ax_5a.semilogy(t_pts, np.abs(E_tot_pts), color='black', label=r'$E(t)$')
ax_5a.semilogy(t_pts, E_tot_rel_pts_ode, 
               color='b', label=r'$\Delta E(t)$ RK23')
ax_5a.semilogy(t_pts, E_tot_rel_pts_LF, 
               color='red', label=r'$\Delta E(t)$ Leapfrog')
ax_5a.set_ylim(1.e-10,  5)  # (1.e-12, 5)
ax_5a.set_xlabel(r'$t$')
ax_5a.set_ylabel(r'Energy')
ax_5a.set_title('Change in energy with time')
ax_5a.legend()

fig_5.tight_layout()

## Animation 
We will use these plots for out animations 

In [ ]:
t_start = 0 
t_end = 50
delta_t = .01


t_pts = np.arange(t_start, t_end+delta_t, delta_t)

#parameters 

G =10
m_1 = 1
m_2 = 1

o1 = gravitationalorbits(G, m_1, m_2)

#inital conditions with center of mass velocity = 0 
x10, x_dot10 = 1, -1
y10, y_dot10 = 1, 1
x20, x_dot20 = -(m_1/m_2)* x10, -(m_1/m_2)*x_dot10
y20, y_dot20 = -(m_1/m_2)* y10, -(m_1/m_2)*y_dot10

z0 = [x10, x_dot10, y10, y_dot10
     ,x20, x_dot20, y20, y_dot20]
x1, x_dot1, y1, y_dot1, x2, x_dot2, y2, y_dot2 = o1.leapfrog_solve_ode(t_pts, z0)


#plotting 

fig = plt.figure(figsize = (8,8))
overall_title = 'Simple Gravitational Orbit using LeapFrog'
fig.suptitle(overall_title, va='baseline')

ax =fig.add_subplot(1,1,1)

start,stop = start_stop_indices(t_pts, t_start, t_end)

ax.plot(x1,y1, color ='b', label =r'$m_1$')
ax.plot(x2,y2, color ='r', label =r'$m_2$')
ax.legend()
ax.set_aspect(1)

fig.tight_layout()

In [ ]:
%%capture 
x_min = -3 
x_max = -x_min 
y_min = -3 
y_max = -y_min 

fig_anim = plt.figure(figsize = (8,8), num = 'Double pendulum')
ax_anim = fig_anim.add_subplot(1,1,1)
ax_anim.set_xlim(x_min, x_max)
ax_anim.set_ylim(y_min, y_max)

#assign the first return from plot to pt1_anim and others so we can later change the values 

ln1_anim, = ax_anim.plot(x1, y1, color = 'b', lw= 1)
ln2_anim, = ax_anim.plot(x2, y2, color = 'r' ,lw= 1)

pt1_anim, = ax_anim.plot(x1[0], y1[0], 'o', markersize = 8, color = 'b')
pt2_anim, = ax_anim.plot(x2[0], y2[0], 'o', markersize = 8, color = 'r')

ax_anim.set_aspect(1)
ax_anim.axis('off')
fig_anim.tight_layout()


In [ ]:
def orbit_animate(i):
    """
    This function will create each from, numbered by i. 
    So each i corresponds to a point in the t_pts array with index i. 
    """
    i_skip = 1*i 
    
    pt1_anim.set_data(x1[i_skip], y1[i_skip])
    pt2_anim.set_data(x2[i_skip], y2[i_skip])
    
    return (pt1_anim, pt2_anim)

    

In [ ]:
frame_interval = 10 #time between frames
frame_number = 1001 #number of frames

anim = animation.FuncAnimation(fig_anim, orbit_animate, init_func = None, 
                              frames = frame_number, 
                              interval = frame_interval, 
                              blit = True, 
                              repeat = False)

In [ ]:
HTML(anim.to_jshtml())